In [ ]:
import xml.etree.ElementTree as etree
import cv2
import numpy as np
import os
import random

In [ ]:
def save_crop_with_xml(image_path, xml_path, ratio, end_str):
  start_path = '/content/drive/My Drive/dekhi/new_original/'
  image = cv2.imread(image_path)
  #print(image.shape)

  tree = etree.parse(xml_path)
  root = tree.getroot()

  check = 0
  new_height = image.shape[0]
  new_width = image.shape[1]

  if image.shape[0]!=int(root.find('size').find('height').text):
    new_height = int(root.find('size').find('height').text)
    check = 1
  if image.shape[1]!=int(root.find('size').find('width').text):
    new_width = int(root.find('size').find('width').text)
    check = 1
  
  if check == 1:
    image = cv2.resize(image, (new_width, new_height), interpolation = cv2.INTER_CUBIC)
  
  startY = int(ratio*image.shape[0]/2)
  endY = image.shape[0] - startY
  startX = int(ratio*image.shape[1]/2)
  endX = image.shape[1] - startX
  cropped = image[startY:endY, startX:endX]


  for node in root:
    if node.tag == 'object':
      node.find('bndbox').find('xmin').text = str(int(node.find('bndbox').find('xmin').text) - startX)
      node.find('bndbox').find('ymin').text = str(int(node.find('bndbox').find('ymin').text) - startY)
      node.find('bndbox').find('xmax').text = str(int(node.find('bndbox').find('xmax').text) - startX)
      node.find('bndbox').find('ymax').text = str(int(node.find('bndbox').find('ymax').text) - startY)
      if int(node.find('bndbox').find('xmin').text) < 0:
        node.find('bndbox').find('xmin').text = '0'
      if int(node.find('bndbox').find('ymin').text) < 0:
        node.find('bndbox').find('ymin').text = '0'
      if int(node.find('bndbox').find('xmax').text) < 0:
        node.find('bndbox').find('xmax').text = '0'
      if int(node.find('bndbox').find('ymax').text) < 0:
        node.find('bndbox').find('ymax').text = '0'
      if int(node.find('bndbox').find('xmin').text) > image.shape[1] - 2*startX - 1:
        node.find('bndbox').find('xmin').text = str(image.shape[1] - 2*startX - 1)
      if int(node.find('bndbox').find('xmax').text) > image.shape[1] - 2*startX - 1:
        node.find('bndbox').find('xmax').text = str(image.shape[1] - 2*startX - 1)
      if int(node.find('bndbox').find('ymin').text) > image.shape[0] - 2*startY - 1:
        node.find('bndbox').find('ymin').text = str(image.shape[0] - 2*startY - 1)
      if int(node.find('bndbox').find('ymax').text) > image.shape[0] - 2*startY - 1:
        node.find('bndbox').find('ymax').text = str(image.shape[0] - 2*startY - 1)
  
  cv2.imwrite(start_path + image_path[:-4] + end_str + '.jpg', cropped)
  tree.write(start_path + xml_path[:-4] + end_str + '.xml')



In [ ]:
#save_crop_with_xml('/content/01.jpg', '/content/01.xml', 0.2, 'xx')

In [ ]:
def save_mirror_flip(image_path, xml_path, end_str):
  start_path = '/content/drive/My Drive/dekhi/new_original/'
  image = cv2.imread(image_path)
  #print(image.shape)
  flipped = cv2.flip(image, 1)

  tree = etree.parse(xml_path)
  root = tree.getroot()

  for node in root:
    if node.tag == 'object':
      temp = int(node.find('bndbox').find('xmin').text)
      node.find('bndbox').find('xmin').text = str(int(root.find('size').find('width').text) - int(node.find('bndbox').find('xmax').text))
      node.find('bndbox').find('xmax').text = str(int(root.find('size').find('width').text) - temp)
  
  cv2.imwrite(start_path + image_path[:-4] + end_str + '.jpg', flipped)
  tree.write(start_path + xml_path[:-4] + end_str + '.xml')
      


In [ ]:
#save_mirror_flip('/content/01.jpg', '/content/01.xml', 'yy')

In [ ]:
def save_resize(image_path, xml_path, new_size):
  image = cv2.imread(image_path)
  #scaleX = new_size / image.shape[1]
  #scaleY = new_size / image.shape[0]

  tree = etree.parse(xml_path)
  root = tree.getroot()

  if root.find('size').find('width').text == '0':
    root.find('size').find('width').text = str(image.shape[1])
  if root.find('size').find('height').text == '0':
    root.find('size').find('height').text = str(image.shape[0])

  resized = cv2.resize(image, (new_size, new_size), interpolation = cv2.INTER_CUBIC)

  scaleX = new_size/int(root.find('size').find('width').text)
  scaleY = new_size/int(root.find('size').find('height').text)

  for node in root:
    if node.tag == 'size':
      node.find('width').text = str(new_size)
      node.find('height').text = str(new_size)
    if node.tag == 'object':
      node.find('bndbox').find('xmin').text = str(int(np.round(float(int(node.find('bndbox').find('xmin').text))*scaleX)))
      node.find('bndbox').find('ymin').text = str(int(np.round(float(int(node.find('bndbox').find('ymin').text))*scaleY)))
      node.find('bndbox').find('xmax').text = str(int(np.round(float(int(node.find('bndbox').find('xmax').text))*scaleX)))
      node.find('bndbox').find('ymax').text = str(int(np.round(float(int(node.find('bndbox').find('ymax').text))*scaleY)))
  
  cv2.imwrite(image_path, resized)
  tree.write(xml_path)


In [ ]:
def improve_contrast_image_using_clahe(bgr_image: np.array) -> np.array:
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_planes = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    hsv_planes[2] = clahe.apply(hsv_planes[2])
    hsv = cv2.merge(hsv_planes)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [ ]:
#save_resize('/content/113.jpg', '/content/113.xml', 1024)

In [ ]:
# Change directory to Data folder
%cd '/content/drive/My Drive/dekhi/original/'

/content/drive/My Drive/dekhi/original


In [ ]:
path = 'path' #Dataset path
next_path = 'path' #Processed images path

files = os.listdir(path)
files.sort()

#print(len(files))

st = 0
if files[0] == '.ipynb_checkpoints':
  st = 1


for i in range(st, len(files), 2):
  image_path = files[i]
  xml_path = files[i+1]

  if files[i] == '231.jpg':  # ignoring faulty image
    continue
  
  im = cv2.imread(image_path)
  tree = etree.parse(xml_path)

  if files[i].endswith('.jpeg'): #handling different image format
    print('+++')
    cv2.imwrite(next_path + image_path[:-5] + '.jpg', im)
  else:
    cv2.imwrite(next_path + image_path[:-4] + '.jpg', im)
  tree.write(next_path + xml_path)





In [ ]:
new_files = os.listdir(next_path)
#print(len(new_files))

In [ ]:
%cd '/content/drive/My Drive/dekhi/new_original/'

In [ ]:
path = '/content/drive/My Drive/dekhi/new_original/'
all_files = os.listdir(path)
all_files.sort()
print(all_files)
st = 0
if all_files[0] == '.ipynb_checkpoints':
  st = 1

count = 700

for i in range(13208, len(all_files), 2):
  if all_files[i] == 'Pias (359).jpg' or all_files[i] == 'Pias (360).jpg':
    continue
  cont = random.randint(0,2)
  if cont == 0 and count<700:
    count+=1
    continue
  ratio = random.uniform(0.01, 0.2)
  #print(type(all_files[i]))
  #print(all_files[i])
  #print(type(all_files[i+1]))
  #print(all_files[i+1])
  save_crop_with_xml(all_files[i], all_files[i+1], ratio, 'v01')
  dcn = random.randint(0,100)
  if dcn < 35:
    ratio2 = random.uniform(0.01, 0.2)
    save_crop_with_xml(all_files[i], all_files[i+1], ratio2, 'v02')
  elif dcn >= 35 and dcn < 90:
    save_mirror_flip(all_files[i], all_files[i+1], 'v02')

new_files = os.listdir(path)
print(len(new_files))


In [ ]:
path = '/content/drive/My Drive/dekhi/new_original'
all_files = os.listdir(path)
all_files.sort()
print(all_files)
st = 0
if all_files[0] == '.ipynb_checkpoints':
  st = 1

count = 0

for i in range(st, len(all_files), 2):
  save_resize(all_files[i], all_files[i+1], 1024)

In [ ]:
path = '/content/drive/My Drive/dekhi/new_original/'
all_files = os.listdir(path)
all_files.sort()
print(all_files)
st = 0
if all_files[0] == '.ipynb_checkpoints':
  st = 1

count = 0

for i in range(st, len(all_files), 2):
  if not all_files[i].endswith('.jpg'):
    print(i)
    print(all_files[i])

In [ ]:
path = '/content/drive/My Drive/dekhi/new_original/'
all_files = os.listdir(path)
all_files.sort()
print(all_files)
st = 0
if all_files[0] == '.ipynb_checkpoints':
  st = 1

count = 0

for i in range(st, len(all_files), 2):
  image = cv2.imread(all_files[i])
  tree = etree.parse(all_files[i+1])
  root = tree.getroot()

  if image.shape != (int(root.find('size').find('height').text), int(root.find('size').find('width').text), 3):
    print(i)
    print(all_files[i])

In [ ]:
path = '/content/drive/My Drive/dekhi/new_original/'
all_files = os.listdir(path)
all_files.sort()
print(all_files)
st = 0
if all_files[0] == '.ipynb_checkpoints':
  st = 1

count = 0

for i in range(st, len(all_files), 2):
  image = cv2.imread(all_files[i])
  tree = etree.parse(all_files[i+1])
  root = tree.getroot()

  if image.shape != (1024, 1024, 3):
    print(i)
    print(all_files[i])

In [ ]:
#print(len(all_files))